In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

In [2]:
trimss = pd.read_csv("trimmed.csv")

In [3]:
trimss.drop(columns=trimss.columns[0], axis=1, inplace=True)

In [4]:
trimss

,Id_Paciente,sumHour,saturaciono,temperatura,frecuenciac,sysPres,diasPres,tiposaturac_GafasO2,tiposaturac_VMK,tiposaturac_Optiflow
0,14260524,14,96.0,37.2,110.0,144.0,79.0,False,True,False
1,14260524,15,NaN,NaN,NaN,NaN,NaN,False,False,False
2,14260524,16,NaN,NaN,NaN,NaN,NaN,False,False,False
3,14260524,17,97.0,37.8,110.0,NaN,NaN,False,True,False
4,14260524,18,NaN,NaN,NaN,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...
342280,8631140337,222,NaN,NaN,NaN,NaN,NaN,False,False,False
342281,8631140337,223,NaN,NaN,NaN,NaN,NaN,False,False,False
342282,8631140337,224,NaN,NaN,NaN,NaN,NaN,False,False,False
342283,8631140337,225,96.0,35.7,65.0,110.0,75.0,False,True,False


In [5]:
#put it in a big list
trimlist = []
for patient in trimss.Id_Paciente.unique():
    trimlist.append(trimss[trimss["Id_Paciente"]==patient])


In [6]:
#RUN IN PREVIOUS SETUP
# import warnings
# warnings.filterwarnings('ignore')
# for i,table in enumerate(trimlist):
#     print(i/len(trimlist))
#     for hour in table["sumHour"].unique():
#         ## FIX INCONSISTENT METRICS (eg. saturaciono)
#         # we've already dont the pressure fix by this point, so we could just get an average for every column that's
#         # numeric past this point
#         if len(table[table["sumHour"]==hour]) >= 2: #if this hour has at least 2 rows
#             for col in table[table["sumHour"]==hour].columns.values:
#                 if table[col].dtype == 'float64':
#                     # print(i)
#                     trimlist[i].loc[trimlist[i]["sumHour"] == hour, col] = trimlist[i].loc[trimlist[i]["sumHour"] == hour, col].mean() #should work??
#     trimlist[i] = trimlist[i].drop_duplicates()



In [7]:
trimlist[0]

,Id_Paciente,sumHour,saturaciono,temperatura,frecuenciac,sysPres,diasPres,tiposaturac_GafasO2,tiposaturac_VMK,tiposaturac_Optiflow
0,14260524,14,96.0,37.2,110.0,144.0,79.0,False,True,False
1,14260524,15,NaN,NaN,NaN,NaN,NaN,False,False,False
2,14260524,16,NaN,NaN,NaN,NaN,NaN,False,False,False
3,14260524,17,97.0,37.8,110.0,NaN,NaN,False,True,False
4,14260524,18,NaN,NaN,NaN,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...
231,14260524,245,NaN,NaN,NaN,NaN,NaN,False,False,False
232,14260524,246,NaN,NaN,NaN,NaN,NaN,False,False,False
233,14260524,247,NaN,NaN,NaN,NaN,NaN,False,False,False
234,14260524,248,NaN,NaN,NaN,NaN,NaN,False,False,False


In [8]:
#first, a function to get the numbers that mark percentages of a number range
def get_percent(table,percentlist):
    outlist = []
    outlist.append(min(table.sumHour)) #first, set our startpoint as the min hour for this patient
    modhours = table.sumHour - min(table.sumHour) #zero standardize for easier calculations later
    for percent in percentlist: #for each percent requested in this array/list/whatever, get the number that corresponds
        outlist.append(round(max(modhours) * percent)+min(table.sumHour)) #percent fed as decimals, like 0.25
    outlist.append(max(table.sumHour)) #cap off
    return outlist

In [9]:
plist = [0.25,0.5,0.75]

In [10]:
get_percent(trimlist[0],plist)

[14, 73, 132, 190, 249]

In [14]:
# brutish, but fast
contVars = ["saturaciono","temperatura","frecuenciac","sysPres","diasPres"]
catVars = ["tiposaturac_GafasO2","tiposaturac_VMK","tiposaturac_Optiflow"]

In [38]:
#next, a function to use that output to get averages for each column and range, and divide that into the amount of rows for the
#next table
def compress(table,percentlist):
    marks = get_percent(table,percentlist)
    outlist = []

    for i,mark in enumerate(marks):
        if i > 0: #skip first entry
            linedict = dict()
            linedict["Id_Paciente"] = table.Id_Paciente[0] #doesnt change so doesnt matter
            filter = np.where((table['sumHour']>=marks[i-1]) & (table['sumHour']<=marks[i]))
            temptable = table.loc[filter]
            for label in contVars:
                linedict[label] = round(temptable[label].mean(),1)
            outlist.append(linedict)
                
    return pd.DataFrame.from_dict(outlist)

In [39]:
compress(trimlist[0],plist)

,Id_Paciente,saturaciono,temperatura,frecuenciac,sysPres,diasPres
0,14260524,95.3,37.3,102.3,121.6,74.4
1,14260524,92.9,37.0,92.4,120.6,72.3
2,14260524,95.7,36.4,89.3,120.8,72.2
3,14260524,93.9,36.6,100.9,130.4,82.6


In [11]:
trimlist[0]

,Id_Paciente,sumHour,saturaciono,temperatura,frecuenciac,sysPres,diasPres,tiposaturac_GafasO2,tiposaturac_VMK,tiposaturac_Optiflow
0,14260524,14,96.0,37.2,110.0,144.0,79.0,False,True,False
1,14260524,15,NaN,NaN,NaN,NaN,NaN,False,False,False
2,14260524,16,NaN,NaN,NaN,NaN,NaN,False,False,False
3,14260524,17,97.0,37.8,110.0,NaN,NaN,False,True,False
4,14260524,18,NaN,NaN,NaN,NaN,NaN,False,False,False
...,...,...,...,...,...,...,...,...,...,...
231,14260524,245,NaN,NaN,NaN,NaN,NaN,False,False,False
232,14260524,246,NaN,NaN,NaN,NaN,NaN,False,False,False
233,14260524,247,NaN,NaN,NaN,NaN,NaN,False,False,False
234,14260524,248,NaN,NaN,NaN,NaN,NaN,False,False,False


In [ ]:
# pd.concat(trimlist).to_csv("trimmed.csv")

In [ ]:
# lab = pd.read_excel('laboratorio_final_new.xlsx') #Other, weirder long data?
pac = pd.read_excel('pac_covid.xlsx') #Patient basic info, ONE ROW PER PATIENT
pac["Dead"] = pac["fecha_fallece"].notna()
pac

In [12]:
ctes_pivot = pd.pivot_table(ctes2,
               values="Valor__form_cte_",
               index="Id_Paciente",
               columns=["Etiqueta__form_cte_","Days","hora"],
               aggfunc="first")

Maybe we should standardize the times here before we go on to stitch them together, so instead of Measurement_Date_Hour_Minute, its standardized to be How much time since entry, or even just to only how many hours since, and we keep it to that.

In [13]:
ctes_pivot.columns = ctes_pivot.columns.to_flat_index()  # Converts MultiIndex to tuples
# ctes_pivot.columns = ['_'.join(col) for col in ctes_pivot.columns]  # Join tuples
ctes_pivot

,"(EscalaDolor, 0 days 00:00:00, 0)","(EscalaDolor, 0 days 00:00:00, 1)","(EscalaDolor, 0 days 00:00:00, 2)","(EscalaDolor, 0 days 00:00:00, 3)","(EscalaDolor, 0 days 00:00:00, 4)","(EscalaDolor, 0 days 00:00:00, 5)","(EscalaDolor, 0 days 00:00:00, 6)","(EscalaDolor, 0 days 00:00:00, 7)","(EscalaDolor, 0 days 00:00:00, 8)","(EscalaDolor, 0 days 00:00:00, 9)",...,"(valorimc, 22 days 00:00:00, 19)","(valorimc, 24 days 00:00:00, 13)","(valorimc, 25 days 00:00:00, 19)","(valorimc, 25 days 00:00:00, 20)","(valorimc, 28 days 00:00:00, 16)","(valorimc, 31 days 00:00:00, 15)","(valorimc, 33 days 00:00:00, 21)","(valorimc, 34 days 00:00:00, 16)","(valorimc, 42 days 00:00:00, 18)","(valorimc, 70 days 00:00:00, 18)"
Id_Paciente,,,,,,,,,,,,,,,,,,,,,
11711053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14260524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14409908,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
22329353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
23608324,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9245104453,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9278045053,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9283692237,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [16]:
joined_1 = pac.join(ctes_pivot, on="Id_Paciente", how="left")
joined_1

,Id_Paciente,Desc_Prueba,Fecha_validaci_n_prueba,Episodio___nico_,Sexo,Edad_a_os__actual_,Fecha__ingreso_,Fecha__alta_,fecha_fallece,dias_hasta_ingreso,...,"(talla (cm), 24 days 00:00:00, 13)","(talla (cm), 25 days 00:00:00, 19)","(talla (cm), 25 days 00:00:00, 20)","(talla (cm), 28 days 00:00:00, 16)","(talla (cm), 31 days 00:00:00, 15)","(talla (cm), 33 days 00:00:00, 21)","(talla (cm), 34 days 00:00:00, 16)","(talla (cm), 42 days 00:00:00, 18)","(talla (cm), 45 days 00:00:00, 18)","(talla (cm), 70 days 00:00:00, 18)"
0,11711053,Coronavirus 2019-nCoV (ARN),2020-03-16,66755125,Hombre,86,2020-03-16,2020-03-21,2020-03-21,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,14260524,Coronavirus 2019-nCoV (ARN),2020-04-07,66844681,Mujer,97,2020-04-07,2020-04-17,NaT,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,14409908,Coronavirus 2019-nCoV (ARN),2020-03-31,66822847,Mujer,94,2020-04-01,2020-04-04,2020-04-04,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,22329353,Coronavirus 2019-nCoV (ARN),2020-09-24,67973760,Mujer,91,2020-10-01,2020-10-09,NaT,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,23608324,Coronavirus 2019-nCoV (ARN),2020-03-13,66745194,Hombre,90,2020-03-13,2020-03-21,NaT,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1637,9245104453,Coronavirus 2019-nCoV (ARN),2021-04-13,69499309,Mujer,50,2021-04-17,2021-04-20,NaT,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1638,9278045053,Coronavirus 2019-nCoV (ARN),2020-03-23,66783136,Mujer,34,2020-03-23,2020-03-26,NaT,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1639,9283692237,Coronavirus-SARS-2 Ag (ICT),2021-01-06,68713110,Mujer,63,2021-01-12,2021-01-19,NaT,6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1640,9294732495,Coronavirus 2019-nCoV (ARN),2021-01-19,68769742,Mujer,32,2021-01-19,2021-01-20,NaT,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# joined_1.dtypes
from sklearn.compose import make_column_selector as selector

categorical_columns_selector = selector(dtype_include=object)
categorical_columns = categorical_columns_selector(joined_1)
categorical_columns

['Desc_Prueba',
 'Sexo',
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 0),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 1),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 2),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 3),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 4),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 5),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 6),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 7),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 8),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 9),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 10),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 11),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 12),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 13),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 14),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 15),
 ('Cantidad de Sat', Timedelta('0 days 00:00:00'), 16),
 ('Cantidad de Sat', Timedelta('0